In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [50]:
np.random.seed(42)

# Simulando o DataFrame base
n_samples = 500

data = {
    'area_m2': np.random.randint(40, 400, n_samples),
    'quartos': np.random.randint(1, 6, n_samples),
    'idade_casa': np.random.randint(0, 50, n_samples),
}

preco = (
    100000 +
    (data['area_m2'] * 3500) +
    (data['quartos'] * 25000) -
    (data['idade_casa'] * 2000) +
    np.random.normal(0, 15000, n_samples)
)

data['preco'] = preco.round(2)

df_housing = pd.DataFrame(data)

# -------------------------
# SUJANDO O DATAFRAME
# -------------------------

# Convertendo tudo para object (simulando CSV problemático)
df_housing = df_housing.astype(object)

# Inserindo valores faltantes aleatórios
for col in df_housing.columns:
    idx = np.random.choice(df_housing.index, size=15, replace=False)
    df_housing.loc[idx, col] = np.nan

# Inserindo strings onde deveriam ser números
df_housing.loc[3, 'area_m2'] = "85m2"
df_housing.loc[10, 'area_m2'] = "unknown"
df_housing.loc[15, 'quartos'] = "three"
df_housing.loc[22, 'idade_casa'] = "old"
df_housing.loc[30, 'preco'] = "R$ 750000"

# Inserindo valores absurdos
df_housing.loc[40, 'area_m2'] = -150
df_housing.loc[55, 'quartos'] = 20
df_housing.loc[70, 'idade_casa'] = 180
df_housing.loc[85, 'preco'] = -300000

# Misturando separador decimal (simulando padrão europeu)
df_housing.loc[100, 'preco'] = "450000,50"

# Criando duplicatas
df_housing = pd.concat([df_housing, df_housing.iloc[[5, 8]]], ignore_index=True)

df_housing.to_excel('residencia.xlsx')

print(df_housing.head(60))

    area_m2 quartos idade_casa       preco
0       142       3         39   586352.14
1       388       4          7  1506730.83
2       310       4         38  1222797.41
3      85m2       2         39   587707.93
4       111       3         13    512272.7
5       228       4         31   909621.02
6        60     NaN         37   247910.54
7       142       1         32   569904.87
8       161       5         22   739994.59
9       254       3         14   1023928.2
10  unknown       3         32  1390908.26
11      127       5         24   605736.32
12      139       4         16   625494.83
13      399       3         32  1489389.06
14      191       1         46   685441.33
15      170   three          1   728239.35
16      189       2         13   783014.15
17      348       3         39  1302616.23
18      297       4         39  1196145.84
19      383       5         38  1479541.55
20      333       5          5         NaN
21      231       4          5  1022816.63
22      316

In [ ]:
#Limpando o DataFrame

df_housing['area_m2'] = pd.to_numeric(df_housing['area_m2'], errors='coerce')
df_housing['quartos'] = pd.to_numeric(df_housing['quartos'], errors='coerce')           #Transformando valores que não numéricos em NaN
df_housing['idade_casa'] = pd.to_numeric(df_housing['idade_casa'], errors='coerce')
df_housing['preco'] = pd.to_numeric(df_housing['preco'], errors='coerce')

df_housing = df_housing.dropna()                                                        #Removendo linhas com valores NaN do DataFrame

print(df_housing.head())

   area_m2  quartos  idade_casa       preco
0    142.0      3.0        39.0   586352.14
1    388.0      4.0         7.0  1506730.83
2    310.0      4.0        38.0  1222797.41
4    111.0      3.0        13.0   512272.70
5    228.0      4.0        31.0   909621.02


In [23]:
#Definindo Variáveis

X = df_housing.drop('preco', axis=1)
y = df_housing['preco']

conversor_polinomial = PolynomialFeatures(degree=2, include_bias=False)
X_poli = conversor_polinomial.fit_transform(X)

In [30]:
#Treinando o modelo

X_treino, X_teste, y_treino, y_teste = train_test_split(X_poli, y, test_size=0.3, random_state=42)

modelo = LinearRegression()
modelo.fit(X_treino,y_treino)
previsoes = modelo.predict(X_teste)

In [ ]:
#Testando o modelo

print(round(mean_absolute_error(previsoes, y_teste)),2)
print(round(root_mean_squared_error(previsoes, y_teste)),2)

#Dados mostrados no terminal correspondem ao erro médio do modelo

12548 2
15423 2


In [ ]:
#testando modelo com dados informados pelo usuário

aream2 = float(input('Digite aqui o tamanho da casa em m2: '))
quartos = float(input('Digite aqui quantos quartos tem na casa: '))
idade = float(input('Digite aqui a idade da casa: '))

dados_usuario = [aream2, quartos, idade]

dados_ficticios = [dados_usuario]
dados_transformados = conversor_polinomial.transform(dados_ficticios)
previsao = modelo.predict(dados_transformados).round(2)

print(previsao)

[637385.21]


/home/davi-dos-santos-freire/VSCode/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
